In [ ]:
pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 1.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.9/208.9

In [ ]:
import pandas as pd
from datasets import load_dataset
data = load_dataset("opus_books", "en-fr")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

In [ ]:
print(data)

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 127085
    })
})


In [ ]:
sample = data["train"][0]
print('Sample data fromthe training data:', sample)


Sample data fromthe training data: {'id': '0', 'translation': {'en': 'The Wanderer', 'fr': 'Le grand Meaulnes'}}


In [ ]:
print(type(data))


<class 'datasets.dataset_dict.DatasetDict'>


In [ ]:
df = data['train']


In [ ]:
df = df.to_pandas()


In [ ]:
df = pd.DataFrame(df['translation'].tolist())


In [ ]:
translations = []
for item in data:
    if isinstance(item, dict) and 'translation' in item:
        en_text = item['translation'].get('en', '')
        fr_text = item['translation'].get('fr', '')
        translations.append({'en': en_text, 'fr': fr_text})
    else:
        print(f"Item is not in expected format: {item}")

Item is not in expected format: train


In [ ]:
df.columns = ['en', 'fr']


In [ ]:
print(df.head())


               en                 fr
0    The Wanderer  Le grand Meaulnes
1  Alain-Fournier     Alain-Fournier
2      First Part    PREMIÈRE PARTIE
3               I   CHAPITRE PREMIER
4     THE BOARDER    LE PENSIONNAIRE


In [ ]:
df.to_csv('df.csv', index=False)

In [ ]:
df.head()

,en,fr
0,The Wanderer,Le grand Meaulnes
1,Alain-Fournier,Alain-Fournier
2,First Part,PREMIÈRE PARTIE
3,I,CHAPITRE PREMIER
4,THE BOARDER,LE PENSIONNAIRE


In [ ]:
df.rename(columns={'en': 'English', 'fr': 'French'}, inplace=True)
df.head()

,English,French
0,The Wanderer,Le grand Meaulnes
1,Alain-Fournier,Alain-Fournier
2,First Part,PREMIÈRE PARTIE
3,I,CHAPITRE PREMIER
4,THE BOARDER,LE PENSIONNAIRE


In [ ]:
import numpy as np
import string
from numpy import array, argmax, random, take
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, RepeatVector
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, GRU, SimpleRNN
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences



In [ ]:
#Removing punctuations and converting text to lowercase for both languages
df['English'] = df['English'].str.replace('[{}]'.format(string.punctuation), '').str.lower()
df['French'] = df['French'].str.replace('[{}]'.format(string.punctuation), '').str.lower()

In [ ]:
# Find indices of empty rows in both languages
english_empty_indices = df[df['English'].str.strip().astype(bool) == False].index
french_empty_indices = df[df['French'].str.strip().astype(bool) == False].index

# Combine indices from both languages to remove empty rows
remove_indices = list(set(english_empty_indices) | set(french_empty_indices))

# Removing empty rows
df.drop(remove_indices, inplace=True)

# Reset indices
df.reset_index(drop=True, inplace=True)


In [ ]:
df

,English,French
0,the wanderer,le grand meaulnes
1,alain-fournier,alain-fournier
2,first part,première partie
3,i,chapitre premier
4,the boarder,le pensionnaire
...,...,...
127080,"therese took the glass, half emptied it, and h...","thérèse prit le verre, le vida à moitié et le ..."
127081,the result was like lightning. the couple fell...,"ce fut un éclair, ils tombèrent l'un sur l'aut..."
127082,the mouth of the young woman rested on the sca...,"la bouche de la jeune femme alla heurter, sur ..."
127083,"the corpses lay all night, spread out contorte...",les cadavres restèrent toute la nuit sur le ca...


In [ ]:
# Importing necessary libraries
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Initialize Tokenizer for English subtitles
tokenizer_english = Tokenizer()
tokenizer_english.fit_on_texts(df['English'])
tokenizer_english

In [ ]:
# Convert text to sequences of integers for English subtitles
sequences_english = tokenizer_english.texts_to_sequences(df['English'])
sequences_english

[[1, 12900],
 [36202, 36203],
 [105, 232],
 [6],
 [1, 17153],
 [9, 551, 17, 82, 395, 21, 5, 2393, 2, 3132, 20903],
 [6, 124, 125, 15741, 395, 57, 814, 1, 152, 43, 294, 4240, 4, 84],
 [42,
  144,
  10,
  232,
  2,
  1,
  375,
  718,
  1199,
  315,
  963,
  3,
  138,
  443,
  113,
  109,
  123,
  4,
  11],
 [42, 37, 637, 7, 1, 2111, 2, 1, 2075, 10088, 6078, 17, 4058, 36204, 1008],
 [22,
  299,
  221,
  6,
  515,
  4,
  530,
  204,
  3948,
  16,
  74,
  94,
  4818,
  8,
  167,
  2,
  1,
  687,
  1008,
  3,
  292,
  2,
  1,
  2075,
  10088,
  6078,
  121,
  4818,
  1727,
  18,
  1,
  12187,
  28381,
  3507],
 [22, 300, 2637, 1, 36205],
 [17,
  1,
  1565,
  246,
  2,
  1,
  410,
  564,
  5,
  122,
  461,
  152,
  14,
  276,
  605,
  1233,
  3,
  5,
  5375,
  20904,
  75,
  91,
  800,
  45,
  1002,
  4741,
  14,
  20905,
  5,
  36206,
  3,
  5,
  1194,
  13677,
  21,
  1,
  244,
  349,
  285,
  1,
  938,
  21,
  1,
  964,
  244,
  5,
  410,
  1218,
  973,
  21,
  1,
  432,
  2275,
  4,
  1,


In [ ]:
# Initialize Tokenizer for French subtitles
tokenizer_french = Tokenizer()
tokenizer_french.fit_on_texts(df['French'])
tokenizer_french

In [ ]:
# Convert text to sequences of integers for French subtitles
sequences_french = tokenizer_french.texts_to_sequences(df['French'])
sequences_french

[[4, 126, 758],
 [48708, 48709],
 [365, 325],
 [706, 327],
 [4, 13611],
 [7, 775, 170, 33, 10, 2224, 1, 3083, 48710, 472],
 [9, 3411, 5, 103, 66, 170, 33, 50, 47, 6, 3, 203, 13, 33, 30609, 32],
 [33, 552, 1223, 4, 477, 167, 301, 740, 311, 2, 33, 539, 9185, 715, 104],
 [33, 36984, 8, 3506, 22, 908, 2121, 1, 1686, 7595],
 [49,
  294,
  6,
  36985,
  99,
  3557,
  44,
  8,
  207,
  3130,
  64,
  4868,
  5,
  3,
  112,
  4,
  908,
  2121,
  69,
  633,
  5438,
  4,
  8327,
  48711,
  2,
  4,
  908,
  543],
 [60, 320, 214, 3, 234, 1657],
 [14,
  598,
  203,
  563,
  37,
  230,
  1233,
  30610,
  100,
  16,
  17679,
  11177,
  5,
  8328,
  22,
  4006,
  14,
  611,
  1080,
  37,
  48712,
  2,
  30611,
  12,
  4062,
  11,
  178,
  29,
  4,
  1148,
  36,
  10,
  126,
  3882,
  29,
  4,
  222,
  678,
  3,
  307,
  69,
  752,
  14,
  234,
  2902,
  2,
  12,
  5217,
  123,
  3,
  2354,
  5,
  119,
  4952,
  27,
  738,
  2,
  36,
  444,
  16,
  1497,
  16,
  3558,
  2,
  16,
  7829,
  12,
  23351,
 

In [ ]:
# Pad sequences to ensure uniform length
max_length = 100

In [ ]:
# Define the maximum sequence length
sequences_english = pad_sequences(sequences_english, maxlen=max_length, padding='post')
sequences_french = pad_sequences(sequences_french, maxlen=max_length, padding='post')

In [ ]:
# Verify the vocabulary sizes
vocab_size_english = len(tokenizer_english.word_index) + 1
vocab_size_french = len(tokenizer_french.word_index) + 1
print("Vocabulary size for English subtitles:", vocab_size_english)
print("Vocabulary size for French subtitles:", vocab_size_french)

Vocabulary size for English subtitles: 60937
Vocabulary size for French subtitles: 83398


In [ ]:
english_length = sequences_english.shape[1]
# Length of English sequences
french_length = sequences_french.shape[1]
# Length of French sequences
print(english_length, french_length)

100 100


In [ ]:
from sklearn.model_selection import train_test_split
# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(sequences_english[:50000], sequences_french[:50000], test_size=0.2, random_state=42)
# Verify the shapes of the datasets
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of y_val:", y_val.shape)


Shape of X_train: (40000, 100)
Shape of y_train: (40000, 100)
Shape of X_val: (10000, 100)
Shape of y_val: (10000, 100)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, RepeatVector
model = Sequential()
model.add(Embedding(input_dim=vocab_size_english, output_dim=128,input_shape=(english_length,), mask_zero=True))
model.add(LSTM(units=512))
model.add(RepeatVector(n=french_length))
model.add(LSTM(units=512, return_sequences=True))
model.add(Dense(units=vocab_size_french, activation='softmax'))


In [ ]:
from tensorflow.keras.optimizers import RMSprop
rms = RMSprop(learning_rate=0.001)
model.compile(optimizer=rms, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=128)

model.summary()


Epoch 1/10
313/313 [==============================] - 1748s 6s/step - loss: 2.2868 - accuracy: 0.7677 - val_loss: 1.9264 - val_accuracy: 0.7706
Epoch 2/10
313/313 [==============================] - 1754s 6s/step - loss: 1.8820 - accuracy: 0.7724 - val_loss: 1.8541 - val_accuracy: 0.7717
Epoch 3/10
313/313 [==============================] - 1756s 6s/step - loss: 1.8282 - accuracy: 0.7734 - val_loss: 1.7919 - val_accuracy: 0.7732
Epoch 4/10
313/313 [==============================] - 1758s 6s/step - loss: 1.8046 - accuracy: 0.7734 - val_loss: 1.8015 - val_accuracy: 0.7727
Epoch 5/10
313/313 [==============================] - 1759s 6s/step - loss: 1.7907 - accuracy: 0.7738 - val_loss: 1.7770 - val_accuracy: 0.7733
Epoch 6/10
313/313 [==============================] - 1760s 6s/step - loss: 1.7803 - accuracy: 0.7740 - val_loss: 1.7722 - val_accuracy: 0.7735
Epoch 7/10
313/313 [==============================] - 1759s 6s/step - loss: 1.7732 - accuracy: 0.7740 - val_loss: 1.7885 - val_accuracy:

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, GRU, Embedding, RepeatVector
from tensorflow.keras.optimizers import RMSprop

model = Sequential()
model.add(Embedding(input_dim=vocab_size_english, output_dim=128, input_shape=(english_length,), mask_zero=True))
model.add(GRU(units=512))
model.add(RepeatVector(n=french_length))
model.add(GRU(units=512, return_sequences=True))
model.add(Dense(units=vocab_size_french, activation='softmax'))



In [ ]:

rms = RMSprop(learning_rate=0.001)
model.compile(optimizer=rms, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=128)

model.summary()

Epoch 1/10
313/313 [==============================] - 1683s 5s/step - loss: 2.2223 - accuracy: 0.7677 - val_loss: 1.8801 - val_accuracy: 0.7701
Epoch 2/10
313/313 [==============================] - 1676s 5s/step - loss: 1.8808 - accuracy: 0.7723 - val_loss: 1.8854 - val_accuracy: 0.7729
Epoch 3/10
313/313 [==============================] - 1676s 5s/step - loss: 1.8478 - accuracy: 0.7729 - val_loss: 2.0496 - val_accuracy: 0.7704
Epoch 4/10
313/313 [==============================] - 1678s 5s/step - loss: 1.8314 - accuracy: 0.7731 - val_loss: 1.7836 - val_accuracy: 0.7728
Epoch 5/10
313/313 [==============================] - 1678s 5s/step - loss: 1.8176 - accuracy: 0.7733 - val_loss: 1.8229 - val_accuracy: 0.7730
Epoch 6/10
313/313 [==============================] - 1678s 5s/step - loss: 1.8070 - accuracy: 0.7733 - val_loss: 1.7889 - val_accuracy: 0.7734
Epoch 7/10
313/313 [==============================] - 1679s 5s/step - loss: 1.8005 - accuracy: 0.7735 - val_loss: 1.7881 - val_accuracy:

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Embedding, RepeatVector
from tensorflow.keras.optimizers import RMSprop

model = Sequential()
model.add(Embedding(input_dim=vocab_size_english, output_dim=128, input_shape=(english_length,), mask_zero=True))
model.add(SimpleRNN(units=512))
model.add(RepeatVector(n=french_length))
model.add(SimpleRNN(units=512, return_sequences=True))
model.add(Dense(units=vocab_size_french, activation='softmax'))


In [ ]:

rms = RMSprop(learning_rate=0.001)
model.compile(optimizer=rms, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=128)

model.summary()


Epoch 1/10
313/313 [==============================] - 1474s 5s/step - loss: 3.7617 - accuracy: 0.7072 - val_loss: 2.2985 - val_accuracy: 0.7687
Epoch 2/10
313/313 [==============================] - 1476s 5s/step - loss: 4.6771 - accuracy: 0.6527 - val_loss: 2.3610 - val_accuracy: 0.7686
Epoch 3/10
313/313 [==============================] - 1484s 5s/step - loss: 2.4733 - accuracy: 0.7588 - val_loss: 2.1977 - val_accuracy: 0.7691
Epoch 4/10
313/313 [==============================] - 1483s 5s/step - loss: 2.2189 - accuracy: 0.7697 - val_loss: 2.3416 - val_accuracy: 0.7692
Epoch 5/10
313/313 [==============================] - 1483s 5s/step - loss: 2.3948 - accuracy: 0.7623 - val_loss: 2.2088 - val_accuracy: 0.7683
Epoch 6/10
313/313 [==============================] - ETA: 0s - loss: 2.2942 - accuracy: 0.7651